In [20]:
import codecs
import re
import pandas as pd
import numpy as np

file='CancerRegistry_clean.csv'

df=pd.read_csv(file)

def remove_mrns_that_arent_numbers(mrn):
    try:
        pattern=r'[A-z]'
        search=re.search(pattern,mrn)
        if search:
            return np.nan
        else:
            return mrn
    except:
        return mrn

df['mrn']=df['mrn'].apply(remove_mrns_that_arent_numbers)
df.dropna(inplace=True, subset=['mrn'])
df['mrn']=df['mrn'].astype(int)


mrns_registry=df.mrn.unique()

df_demo=pd.read_csv('ENT_CA_DM_Cohort.txt',delimiter='|')
mrns_cohort=df_demo.mrn.unique()
print ('mrns in registry:',len(mrns_registry))
print ('mrns in cohort',len(mrns_cohort))

df_overlap=df[df['mrn'].isin(mrns_cohort)]

print('number of patients in cohort with info in registry:',len(df_overlap.mrn.unique()))
#df_overlap.to_csv('examine.csv')

mrns in registry: 1919
mrns in cohort 2664
number of patients in cohort with info in registry: 1919


C:\Users\12488\python\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
df_DB=pd.read_csv('RM_Database.csv')
print (df_DB.columns)
cols=['mrn','date_of_primary_diagnosis']
df_DB=df_DB[cols]
df_DB.dropna(inplace=True,subset=['date_of_primary_diagnosis'])
print (df_DB)

df_to_add=df[['mrn','date_of_initial_diagnosis']]
df_to_add.drop_duplicates(keep='first',subset=['date_of_initial_diagnosis'],inplace=True)
print (df_to_add)

Index(['mrn', 'pt_name', 'gender', 'race', 'ethnicity', 'date_of_birth',
       'date_of_death', 'drinks_per_week', 'TOBACCO_USER', 'pack_years',
       'Stage', 'TNM', 'T', 'N', 'M', 'narrative', 'Dx_',
       'date_of_primary_diagnosis', 'primary_diagnoses', 'secondary_diagnoses',
       'HPV_status'],
      dtype='object')
          mrn date_of_primary_diagnosis
0      218087                2011-03-15
1      218644                2010-11-03
2      255332                2004-06-10
3      263405                2012-10-17
4      296701                2011-07-15
...       ...                       ...
2655  3938435                2020-02-12
2656  3938622                2020-02-12
2658  3939715                2020-02-11
2659  3940931                2020-02-18
2660  3942512                2020-02-21

[2512 rows x 2 columns]
          mrn date_of_initial_diagnosis
0     2685854                03/29/2010
2     2362656                09/10/1998
5     2759491                07/26/2004
8     2

C:\Users\12488\python\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [22]:
df_merge=df_DB.merge(df_to_add,on='mrn',how='inner')
df_merge.columns=['mrn','date_dx_DB','date_dx_registry']
print (df_merge)
df_merge['date_dx_DB']=pd.to_datetime(df_merge['date_dx_DB'])
df_merge['date_dx_registry']=pd.to_datetime(df_merge['date_dx_registry'])


diffs=[]
for index,row in df_merge.iterrows():
    diff=row['date_dx_DB']-row['date_dx_registry']
    diffs.append(abs(diff.days))
df_merge['diff']=diffs

df_merge=df_merge[df_merge['date_dx_registry']>='2000']

print (df_merge.describe())
#df_merge.to_csv('examine.csv',index=False)

          mrn  date_dx_DB date_dx_registry
0      218087  2011-03-15       03/02/2011
1      218644  2010-11-03       11/04/2010
2      296701  2011-07-15       06/30/2011
3      372090  2007-04-17       02/26/1999
4      421991  2012-08-07       06/29/2012
...       ...         ...              ...
1570  3867621  2019-05-29       05/31/2019
1571  3868198  2019-06-03       06/03/2019
1572  3874860  2019-06-26       06/26/2019
1573  3875964  2019-06-25       06/25/2019
1574  3879612  2019-07-11       07/11/2019

[1575 rows x 3 columns]
                mrn         diff
count  1.523000e+03  1523.000000
mean   2.968609e+06   306.300066
std    6.679278e+05   735.464795
min    2.180870e+05     0.000000
25%    2.786592e+06    11.000000
50%    3.111036e+06    25.000000
75%    3.374616e+06   183.500000
max    3.879612e+06  7259.000000
